# Working with Text Files

**Module 06 | Notebook 02**

---

## Objective
By the end of this notebook, you will master:
- Reading text/CSV files with np.loadtxt
- Reading with np.genfromtxt for missing data
- Writing arrays to text files
- Handling different delimiters and formats
- Performance considerations

In [ ]:
import numpy as np
import os
np.set_printoptions(precision=2)

---
## 1. np.savetxt() - Writing Text Files

In [ ]:
# Save array to text file
arr = np.arange(12).reshape(3, 4)
print(f"Array to save:\n{arr}")

np.savetxt('array.txt', arr)
print("\nSaved to array.txt")

In [ ]:
# View the file contents
with open('array.txt', 'r') as f:
    print(f.read())

In [ ]:
# Custom delimiter (CSV)
np.savetxt('array.csv', arr, delimiter=',')

with open('array.csv', 'r') as f:
    print("CSV format:")
    print(f.read())

In [ ]:
# Custom format and header
np.savetxt('formatted.csv', arr, 
           delimiter=',',
           fmt='%d',           # Integer format
           header='Col1,Col2,Col3,Col4',
           comments='')        # No # before header

with open('formatted.csv', 'r') as f:
    print(f.read())

In [ ]:
# Format specifiers
floats = np.random.rand(3, 4) * 100

# Different formats
np.savetxt('fmt_default.txt', floats)           # Default scientific
np.savetxt('fmt_fixed.txt', floats, fmt='%.2f') # 2 decimal places
np.savetxt('fmt_int.txt', floats, fmt='%d')     # Integer

print("Fixed point format:")
with open('fmt_fixed.txt', 'r') as f:
    print(f.read())

---
## 2. np.loadtxt() - Reading Text Files

In [ ]:
# Basic loading
loaded = np.loadtxt('array.txt')
print(f"Loaded from txt:\n{loaded}")

In [ ]:
# Load CSV
loaded_csv = np.loadtxt('array.csv', delimiter=',')
print(f"Loaded from CSV:\n{loaded_csv}")

In [ ]:
# Load with header skip
loaded_header = np.loadtxt('formatted.csv', delimiter=',', skiprows=1)
print(f"Skipped header:\n{loaded_header}")

In [ ]:
# Load specific columns
np.savetxt('multicol.csv', np.arange(20).reshape(4, 5), delimiter=',')

# Load only columns 0 and 2
selected = np.loadtxt('multicol.csv', delimiter=',', usecols=(0, 2, 4))
print(f"Selected columns:\n{selected}")

In [ ]:
# Specify dtype
loaded_int = np.loadtxt('array.csv', delimiter=',', dtype=int)
print(f"As integers:\n{loaded_int}")
print(f"Dtype: {loaded_int.dtype}")

In [ ]:
# Unpack to separate arrays
col1, col2, col3, col4 = np.loadtxt('array.csv', delimiter=',', unpack=True)
print(f"Column 1: {col1}")
print(f"Column 2: {col2}")

---
## 3. np.genfromtxt() - Handling Missing Data

In [ ]:
# Create file with missing values
with open('missing.csv', 'w') as f:
    f.write("1,2,3\n")
    f.write("4,,6\n")      # Missing value
    f.write("7,8,\n")      # Missing at end
    f.write(",10,11\n")    # Missing at start

print("File with missing values:")
with open('missing.csv', 'r') as f:
    print(f.read())

In [ ]:
# loadtxt would fail on missing data
# genfromtxt handles it
data = np.genfromtxt('missing.csv', delimiter=',')
print(f"With genfromtxt (NaN for missing):\n{data}")

In [ ]:
# Specify fill value for missing
data = np.genfromtxt('missing.csv', delimiter=',', filling_values=0)
print(f"Fill with 0:\n{data}")

In [ ]:
# Create file with header and mixed types
with open('mixed.csv', 'w') as f:
    f.write("name,age,score\n")
    f.write("Alice,25,95.5\n")
    f.write("Bob,30,87.0\n")
    f.write("Charlie,35,92.5\n")

In [ ]:
# Load with automatic dtype detection
data = np.genfromtxt('mixed.csv', delimiter=',', 
                      names=True,      # First row is header
                      dtype=None,      # Auto-detect types
                      encoding='utf-8')

print(f"Structured array:\n{data}")
print(f"Names: {data['name']}")
print(f"Ages: {data['age']}")

In [ ]:
# Custom converters
with open('dates.csv', 'w') as f:
    f.write("2023-01-15,100\n")
    f.write("2023-02-20,200\n")

# Convert date string to days since 2023-01-01
def date_converter(s):
    parts = s.split(b'-')
    return int(parts[1]) * 30 + int(parts[2])

data = np.genfromtxt('dates.csv', delimiter=',',
                      converters={0: date_converter})
print(f"With date conversion:\n{data}")

---
## 4. Other Text Functions

In [ ]:
# np.fromstring - parse string to array
s = "1 2 3 4 5"
arr = np.fromstring(s, dtype=int, sep=' ')
print(f"From string: {arr}")

In [ ]:
# np.array2string - format array as string
arr = np.array([[1.23456, 2.34567], [3.45678, 4.56789]])
s = np.array2string(arr, precision=2, separator=', ')
print(f"Array as string:\n{s}")

In [ ]:
# np.set_string_function - customize repr (advanced)
# np.fromregex - parse using regex (advanced)

---
## 5. Performance: loadtxt vs genfromtxt

In [ ]:
# Create larger test file
large = np.random.rand(10000, 10)
np.savetxt('large.csv', large, delimiter=',')

import time

# loadtxt
start = time.time()
data1 = np.loadtxt('large.csv', delimiter=',')
loadtxt_time = time.time() - start

# genfromtxt
start = time.time()
data2 = np.genfromtxt('large.csv', delimiter=',')
genfromtxt_time = time.time() - start

print(f"loadtxt: {loadtxt_time:.3f}s")
print(f"genfromtxt: {genfromtxt_time:.3f}s")
print(f"loadtxt is ~{genfromtxt_time/loadtxt_time:.1f}x faster")

In [ ]:
# For very large files, consider pandas
# pandas.read_csv is highly optimized
# Can convert to numpy: df.values or df.to_numpy()

---
## 6. Common Patterns

In [ ]:
# Pattern: Load numeric data, skip text header
with open('data_with_header.csv', 'w') as f:
    f.write("# This is a comment\n")
    f.write("# Another comment\n")
    f.write("x,y,z\n")
    f.write("1,2,3\n")
    f.write("4,5,6\n")

data = np.loadtxt('data_with_header.csv', delimiter=',', 
                   skiprows=3,    # Skip first 3 lines
                   comments='#')  # Ignore lines starting with #
print(f"Data:\n{data}")

In [ ]:
# Pattern: Read subset of large file
# First 100 rows
subset = np.loadtxt('large.csv', delimiter=',', max_rows=100)
print(f"Subset shape: {subset.shape}")

In [ ]:
# Pattern: Write with row names
data = np.random.rand(3, 4)
row_names = ['Row1', 'Row2', 'Row3']

with open('with_rownames.csv', 'w') as f:
    f.write(',Col1,Col2,Col3,Col4\n')
    for name, row in zip(row_names, data):
        f.write(f"{name},{','.join(f'{x:.3f}' for x in row)}\n")

with open('with_rownames.csv', 'r') as f:
    print(f.read())

---
## Key Points Summary

**Writing:**
- `np.savetxt()`: Arrays to text files
- `fmt` for format, `delimiter` for separator
- `header` and `comments` for metadata

**Reading:**
- `np.loadtxt()`: Simple, fast, strict
- `np.genfromtxt()`: Flexible, handles missing data
- Key params: `delimiter`, `skiprows`, `usecols`, `dtype`

**Best Practices:**
- Use loadtxt for clean data (faster)
- Use genfromtxt for messy/missing data
- Consider pandas for very large CSV files

---
## Interview Tips

**Q1: Difference between loadtxt and genfromtxt?**
> - loadtxt: Faster, but requires clean data with no missing values
> - genfromtxt: Handles missing data, auto-detect dtypes, more flexible

**Q2: How do you handle CSV with header row?**
> `np.loadtxt(..., skiprows=1)` or `np.genfromtxt(..., names=True)`

**Q3: How to read only specific columns?**
> Use `usecols=(0, 2, 4)` parameter with column indices

**Q4: Best approach for large CSV files?**
> Use pandas.read_csv (optimized), then convert to numpy with `.to_numpy()`. For very large files, use chunked reading.

---
## Cleanup

In [ ]:
# Clean up test files
import glob

for f in glob.glob('*.txt') + glob.glob('*.csv'):
    os.remove(f)
    print(f"Removed: {f}")

---
## Next Notebook
**03_binary_and_compressed_files.ipynb** - Working with binary formats and other file types.